In [1]:
import requests
import pandas as pd

Defining the function that gets the data from the API.

In [2]:
def getMapResponse(offset, trainStations = False, **kwargs):
    '''Returns a JSON-reponse object containing limit (default and max is 2000) number of items, starting from offset. trainStations determines whether or not the function returns stations or tracks.'''

    limit = kwargs.get("limit", 2000)
    inputalternativ = kwargs.get("inputalternativ", 1)

    if trainStations:
        #Denne inneholder stasjoner, og ønskes brukt sammen med alt 3. Kan kanskje brukes sammen med 1?
        url = f"https://dvlkart.banenor.no/arcgis/rest/services/Banenettverk/MapServer/2/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Meter&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset={offset}&resultRecordCount={limit}&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
    elif inputalternativ == 1:
        #Denne skal testes mer, inneholder en forenklet strekningsmodell
        url = f"https://dvlkart.banenor.no/arcgis/rest/services/Banenettverk/MapServer/10/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Meter&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset={offset}&resultRecordCount={limit}&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
    elif inputalternativ == 2:
        #Denne inneholder 243 rader, men hver inneholder for lange objekter til å vises korrekt, så de blir kuttet av, og viser bare starten på strekninger.
        url = f"https://dvlkart.banenor.no/arcgis/rest/services/Banelenke/MapServer/0/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset={offset}&resultRecordCount={limit}&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
    elif inputalternativ == 3:
        #Denne inneholder for mange rader, som hver er ganske korte, og blir for mange til at Powerapps kan vise dem korrekt. Den viser bare de første 2000.
        url = f"https://dvlkart.banenor.no/arcgis/rest/services/Banenettverk/MapServer/14/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=Banenavn&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset={offset}&resultRecordCount={limit}&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
    
    payload = {}
    headers = {}
    response = requests.get(url, headers=headers, data=payload)
    #print("URL used: " + url)
    print("Length of API response:", len(response.text))
    return response

Main loop: 
This function fills the dataframe SporDF with all the data from the API.

In [3]:
def fillMapDF(trainStations, offset, limit, sporDF):
    hentetAlt = False
        
    while not hentetAlt:
        SporRawJSON = getMapResponse(offset = offset, trainStations = trainStations).json()  
        
        tempdf = pd.json_normalize(SporRawJSON)
        tempdf = tempdf.explode("features")
        print("Rows of new data:", tempdf.shape[0])
        sporDF = pd.concat([sporDF, tempdf], ignore_index=True)
        #print(sporDF.shape)

        if "exceededTransferLimit" in SporRawJSON:
            if SporRawJSON["exceededTransferLimit"] == True:
                print(f"Transfer limit exceeded. Objects number {offset}-{offset+limit} has been collected. Fetching the next {limit}...")
                offset += limit
        else:
            print("All data collected.")
            hentetAlt = True                
        
                
    print("\n")
    return sporDF

In [4]:
#Sample response
'''
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [13.6299147114066, 66.1564617390492],
          [13.6298764104629, 66.1564525877301],
          [13.6299147114066, 66.1564617390492]
        ]
      },
      "properties": {
        "Banenavn": "Nordlandsbanen"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [5.73934403181721, 60.4593989800107],
          [5.73934270707218, 60.4593853650686],
          [5.73934512720757, 60.4593612516533],
          [5.73932950205534, 60.4592613030111],
          [5.73930664342178, 60.4591609077203],
          [5.73925735839471, 60.4590101723198],
          [5.73919006967426, 60.4588611528486],
          [5.73910527704731, 60.4587143108161],
          [5.73901556669164, 60.4585875023944]
        ]
      },
      "properties": {
        "Banenavn": "Bergensbanen"
      }
    }
  ]
  '''

'\n  "type": "FeatureCollection",\n  "features": [\n    {\n      "type": "Feature",\n      "geometry": {\n        "type": "LineString",\n        "coordinates": [\n          [13.6299147114066, 66.1564617390492],\n          [13.6298764104629, 66.1564525877301],\n          [13.6299147114066, 66.1564617390492]\n        ]\n      },\n      "properties": {\n        "Banenavn": "Nordlandsbanen"\n      }\n    },\n    {\n      "type": "Feature",\n      "geometry": {\n        "type": "LineString",\n        "coordinates": [\n          [5.73934403181721, 60.4593989800107],\n          [5.73934270707218, 60.4593853650686],\n          [5.73934512720757, 60.4593612516533],\n          [5.73932950205534, 60.4592613030111],\n          [5.73930664342178, 60.4591609077203],\n          [5.73925735839471, 60.4590101723198],\n          [5.73919006967426, 60.4588611528486],\n          [5.73910527704731, 60.4587143108161],\n          [5.73901556669164, 60.4585875023944]\n        ]\n      },\n      "properties": 

Define and initialize parameters.

In [5]:
offset = 0
limit = 2000
sporDF = pd.DataFrame(None)
saveFileName = "GeoJSON-spor-forenklet-modell.xlsx"

trainStations = True

In [6]:
sporDF = fillMapDF(trainStations=False, offset = offset, limit = limit, sporDF = sporDF)
sporDF = fillMapDF(trainStations=True, offset = offset, limit = limit, sporDF = sporDF)
print(f"Total number of JSON objects: {sporDF.shape[0]}.")

Length of API response: 141632
Rows of new data: 44
All data collected.


Length of API response: 195080
Rows of new data: 496
All data collected.


Total number of JSON objects: 540.


In [7]:
pd.options.display.max_colwidth = 50 #Default = 50
print(sporDF.tail()) 

                  type                                           features
535  FeatureCollection  {'type': 'Feature', 'id': 1252, 'geometry': {'...
536  FeatureCollection  {'type': 'Feature', 'id': 1651, 'geometry': {'...
537  FeatureCollection  {'type': 'Feature', 'id': 1652, 'geometry': {'...
538  FeatureCollection  {'type': 'Feature', 'id': 1653, 'geometry': {'...
539  FeatureCollection  {'type': 'Feature', 'id': 1654, 'geometry': {'...


Function for extracting label ("name") for each GeoJSON object type, "Banenavn" for track, "Navn" for stations.

In [8]:
def setLabelTrackOrStationTest(row):
    if row["geometry"]["type"] == "LineString":
        #print("inside linestring: ", row.get("properties", None).get("Banenavn", "Ukjent banestrekning"))
        return row.get("properties", "L1").get("Banenavn", "Ukjent banestrekning")
    elif row["geometry"]["type"] == "Point":
        #print("inside point", row.get("properties", None).get("Navn", "Ukjent stasjon"))
        return row.get("properties", "P1").get("Navn", "Ukjent stasjon")
    elif row["geometry"]["type"] == "MultiLineString":
        return row.get("properties", "M1").get("Banenavn", "Ukjent banestrekning")

In [9]:
def makeCircleFromPoint(row):
    if row["geometry"]["type"] == "Point":
        row["properties"]["subType"] = "Circle"
        row["properties"]["radius"] = 75
    return row

Extracting "banenavn" (track name) from properties, and creating a separate column for that. Also adding a a columns where features is converted to string, to enable string.replace-functions. THat is needed to replace ' to ". Also adding a column with len(strfeature) for fun
.

In [10]:
#Purely for debugging the last part
sporDFbackup = sporDF.copy()

In [25]:
##Purely for debugging the last part
sporDF = sporDFbackup.copy()

In [26]:
#sporDF = sporDF.assign(label2 = sporDF["features"].apply(lambda x: x["geometry"]["type"])) 
sporDF.index.name = "index"
sporDF = sporDF.assign(label = sporDF["features"].apply(lambda x: setLabelTrackOrStationTest(x)))
sporDF["features"] = sporDF["features"].apply(lambda x: makeCircleFromPoint(x))

#print(sporDF.loc[sporDF["features"].str.contains("Circle")])

sporDF = sporDF.assign(strfeatures = sporDF["features"].astype("string").str.replace("'", '"') )
sporDF = sporDF.assign(strlen = sporDF["strfeatures"].map(len)) 

#Drops all other columns than the 3(/4) we are interested in.
if "label2" in sporDF.columns:
    sporDF = sporDF[["label", "label2", "strfeatures", "strlen"]]
else:
    sporDF = sporDF[["label", "strfeatures", "strlen"]]


In [27]:
pd.options.display.max_colwidth = 20 #Default = 50

print(sporDF.shape)
print(sporDF.columns)
print(sporDF.sort_values("strlen", ascending=True))
#Check for missing values
errorDF = sporDF.loc[sporDF["label"].isin([None, "B1", "Ukjent banestrekning", "S1", "Ukjent stasjon", "M1"])]
print(errorDF[["strfeatures", "strlen"]])

(540, 3)
Index(['label', 'strfeatures', 'strlen'], dtype='object')
                     label          strfeatures  strlen
index                                                  
41     Tinnosbanen Noto...  {"type": "Featur...     392
28       Ofotbanen Djupvik  {"type": "Featur...     412
29      Ofotbanen Katterat  {"type": "Featur...     415
26     Østfoldbanen Haf...  {"type": "Featur...     426
352                Alnabru  {"type": "Featur...     436
...                    ...                  ...     ...
33              Rørosbanen  {"type": "Featur...   11522
7             Bergensbanen  {"type": "Featur...   13099
8               Dovrebanen  {"type": "Featur...   16537
35           Sørlandsbanen  {"type": "Featur...   18893
19          Nordlandsbanen  {"type": "Featur...   24707

[540 rows x 3 columns]
Empty DataFrame
Columns: [strfeatures, strlen]
Index: []


In [30]:
print(sporDF.iloc[sporDF["strfeatures"].str.contains("Circle").values])

             label          strfeatures  strlen
index                                          
44        Krekling  {"type": "Featur...     444
45         Rudstad  {"type": "Featur...     441
46            Arna  {"type": "Featur...     440
47         Skansen  {"type": "Featur...     443
48          Dombås  {"type": "Featur...     441
...            ...                  ...     ...
535         Orstad  {"type": "Featur...     449
536    Martineåsen  {"type": "Featur...     455
537         Hobekk  {"type": "Featur...     449
538         Myrane  {"type": "Featur...     450
539          Ørvik  {"type": "Featur...     446

[496 rows x 3 columns]


In [267]:
sporDF.to_excel(saveFileName)

Eksempel på json_normalize kode som illustrerer konseptet godt.

In [154]:
#Example json_normalize
data = [
    {
        "state": "Florida",
        "shortname": "FL",
        "info": {"governor": "Rick Scott"},
        "counties": [
            {"name": "Dade", "population": 12345},
            {"name": "Broward", "population": 40000},
            {"name": "Palm Beach", "population": 60000},
        ],
    },
    {
        "state": "Ohio",
        "shortname": "OH",
        "info": {"governor": "John Kasich"},
        "counties": [
            {"name": "Summit", "population": 1234},
            {"name": "Cuyahoga", "population": 1337},
        ],
    },
]
print(pd.DataFrame(data))
result = pd.json_normalize(
    data, "counties", [["info", "governor"]])
print("result DF\n", result)

     state shortname                         info  \
0  Florida        FL   {'governor': 'Rick Scott'}   
1     Ohio        OH  {'governor': 'John Kasich'}   

                        counties  
0  [{'name': 'Dade', 'populat...  
1  [{'name': 'Summit', 'popul...  
result DF
          name  population info.governor
0        Dade       12345    Rick Scott
1     Broward       40000    Rick Scott
2  Palm Beach       60000    Rick Scott
3      Summit        1234   John Kasich
4    Cuyahoga        1337   John Kasich
